In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import random
import string
import json
import re
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/english-french-google-translation/english_french.csv


In [2]:
path = "/kaggle/input/english-french-google-translation/english_french.csv"

In [3]:
df = pd.read_csv('/kaggle/input/english-french-google-translation/english_french.csv',index_col = 0)

In [4]:
df.head()

,English,French
0,We do not have any choice.,Nous n'avons pas le choix.
1,I like that it is soft.,J'aime que ce soit doux.
2,Was there an earthquake?,Y a-t-il eu un tremblement de terre ?
3,They say he is sick.,Elles disent qu'il est malade.
4,You should always wear a seat belt when you ar...,On devrait toujours mettre une ceinture lorsqu...


In [5]:
df.isnull().any()

English    False
French     False
dtype: bool

In [6]:
text_pairs =list(zip(df['English'],df['French']))
random.seed(42)
random.shuffle(text_pairs)
print(text_pairs[:5])

[('I did not want to give up.', 'Je ne voulais pas abandonner.'), ('What is happening?', "Qu'est-ce qui se passe\u202f?"), ('Protesters tried to disrupt the meeting.', 'Les protestataires tentèrent de perturber la réunion.'), ('You could not have picked a better spot.', "Vous n'auriez pas pu choisir un meilleur endroit."), ('That looks like it hurts.', 'On dirait que ça fait mal.')]


In [7]:
num_val_samples = int(0.1 * len(text_pairs))
num_train_samples = len(text_pairs) -  2*num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples+num_val_samples]
test_pairs = text_pairs[num_train_samples+num_val_samples:]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f'{len(test_pairs)} test pairs')

108742 total pairs
86994 training pairs
10874 validation pairs
10874 test pairs


In [8]:
list1 = list(df['English'])
list1 = [word.strip().strip('!').strip('.').strip(',').strip('?').lower() for sentence in list1 for word in sentence.split(' ') if len(word)]

print('approx english vocab:',len(set(list1)))

list2 = list(df['French'])
list2 = [word.strip('!').strip('.').strip(',').strip('?') for sentence in list2 for word in sentence.split(' ')]
print('approx french vocab:',len(set(list2)))

approx english vocab: 16443
approx french vocab: 29092


In [9]:
for i in range(10,20):
    print(list1[i*10:i*10+10])

['a', 'relatively', 'safe', 'neighborhood', 'tom', 'and', 'mary', 'were', 'on', 'the']
['same', 'wavelength', 'kripananda', 'this', 'place', 'is', 'boring', 'what', 'is', 'your']
['favorite', 'toothpaste', 'qualitised', 'postulated', 'complement', 'why', 'do', 'not', 'you', 'tell']
['me', 'what', 'you', 'want', 'to', 'hear', 'someone', 'cleaned', 'my', 'room']
['while', 'i', 'was', 'gone', 'he', 'lived', 'alone', 'in', 'the', 'forest']
['i', 'feel', 'bad', 'that', 'i', 'have', 'not', 'paid', 'you', 'yet']
['are', 'you', 'already', 'married', 'i', 'am', 'going', 'to', 'drive', 'myself']
['it', 'has', 'a', 'leak', 'whether', 'shakespeare', 'wrote', 'this', 'poem', 'or']
['not', 'will', 'probably', 'remain', 'a', 'mystery', 'you', 'are', 'safe', 'here']
['i', 'prefer', 'to', 'travel', 'alone', 'classifies', 'we', 'appreciate', 'your', 'kind']


In [10]:
for i in range(10,20):
    print(list2[i*10:i*10+10])

['vous', 'demander', 'une', 'dernière', 'faveur', 'Je', 'vis', 'dans', 'un', 'quartier']
['relativement', 'sûr', 'Tom', 'et', 'Marie', 'étaient', 'sur', 'la', 'même', 'longueur']
['d’onde', 'Kripananda', 'Cet', 'endroit', 'est', 'ennuyeux', 'Quel', 'est', 'votre', 'dentifrice']
['préféré\u202f', 'qualifié', 'postulé', 'complément', 'Pourquoi', 'ne', 'me', 'dites-vous', 'pas', 'ce']
['que', 'vous', 'voulez', 'entendre', '', "Quelqu'un", 'nettoya', 'ma', 'chambre', 'pendant']
['que', "j'étais", 'parti', 'Il', 'a', 'vécu', 'seul', 'dans', 'la', 'forêt']
['Je', 'me', 'sens', 'mal', 'de', 'ne', 'pas', 'encore', "t'avoir", 'payé']
['Êtes-vous', 'déjà', 'mariée', '', 'Je', 'vais', 'conduire', 'par', 'mes', 'propres']
['moyens', 'Elle', 'fuit', 'Si', 'Shakespeare', 'a', 'écrit', 'ce', 'poème', 'ou']
['pas', 'restera', 'probablement', 'une', 'énigme', 'Vous', 'êtes', 'en', 'sécurité', 'ici']


In [11]:
def tf_lower_and_split_punct(text):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "[^ a-z.?!,¿]", "")
    text = tf.strings.regex_replace(text, "[.?!,¿]", r" \0 ")
    text = tf.strings.strip(text)
    text = tf.strings.join(["[SOS]", text, "[EOS]"], separator=" ")
    return text

In [12]:
#some extra characters in french that are not in english
my_extra_list = ['À', 'Â', 'Ç', 'É', 'Ê', 'Ô', 'à', 'á', 'â', 'ç', 'è', 'é', 'ê', 'ë', 'î', 'ï', 'ô', 'ù', 'û', 'ü', 'ō', 'œ']
my_extra_list = [i.lower() for i in my_extra_list]
my_extra_list = list(set(my_extra_list))

my_extra_string = ''.join(my_extra_list)
print(my_extra_string.lower())

àōœâîûüêëùôáçïéè


In [13]:
def tf_lower_and_split_punct_2(text):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "[^ a-zçèœêâôïûùüàáōëîé.?!,¿]", "")
    text = tf.strings.regex_replace(text, "[.?!,¿]", r" \0 ")
    text = tf.strings.strip(text)
    text = tf.strings.join(["[SOS]", text, "[EOS]"], separator=" ")
    return text

In [14]:
vocab_size = 15000
sequence_length = 20
batch_size = 64

In [15]:
# vectorization
english_vectorization = tf.keras.layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length,
    standardize=tf_lower_and_split_punct
)

french_vectorization = tf.keras.layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length+1,
    standardize=tf_lower_and_split_punct_2
)

train_eng_texts = [pair[0] for pair in train_pairs]
train_fre_texts = [pair[1] for pair in train_pairs]

english_vectorization.adapt(train_eng_texts)
french_vectorization.adapt(train_fre_texts)

In [16]:
#save the vectorization layers

english_vocab = english_vectorization.get_vocabulary()
    
with open('english_vocab.json', 'w', encoding='utf-8') as f:
    json.dump(english_vocab, f)
    
french_vocab = french_vectorization.get_vocabulary()
    
with open('french_vocab.json', 'w', encoding='utf-8') as f:
    json.dump(french_vocab, f)

In [17]:
print(len(english_vocab))
print(english_vocab[:20])
print(french_vocab[:20])

15000
['', '[UNK]', '[SOS]', '[EOS]', '.', 'i', 'you', 'to', 'the', 'is', '?', 'not', 'a', 'do', 'that', 'are', 'it', 'have', 'tom', 'he']
['', '[UNK]', '[SOS]', '[EOS]', '.', 'je', 'de', 'pas', '?', 'que', 'ne', 'à', 'la', 'le', 'vous', 'il', 'tom', 'est', ',', 'a']


In [18]:
list1 = [x for x in english_vocab if len(x)==10]
len(list1)

5606

In [19]:
def format_dataset(eng, spa):
    eng = english_vectorization(eng)
    fre = french_vectorization(spa)
    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": fre[:, :-1],
        },
        fre[:, 1:],
    )
    
def make_dataset(pairs):
    eng_texts, fre_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    fre_texts = list(fre_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, fre_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.cache().shuffle(2048).prefetch(16)

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)
test_ds = make_dataset(test_pairs)

In [21]:
for inp,tar in train_ds.take(1):
    print(inp['encoder_inputs'][:2])
    print(inp['decoder_inputs'][:2])
    print(tar[:2])

tf.Tensor(
[[   2   32    6  332   22  153   10    3    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   2    8 8067 6368   62 1474 2283  104    8 4598 6368    4    3    0
     0    0    0    0    0    0]], shape=(2, 20), dtype=int64)
tf.Tensor(
[[    2   122   504    20   175     8     3     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [    2  7576 11605    17  2661 14986    38  1655     9  7576 11591     4
      3     0     0     0     0     0     0     0]], shape=(2, 20), dtype=int64)
tf.Tensor(
[[  122   504    20   175     8     3     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [ 7576 11605    17  2661 14986    38  1655     9  7576 11591     4     3
      0     0     0     0     0     0     0     0]], shape=(2, 20), dtype=int64)


In [22]:
vocab_size = 15000
units_1 = 128

In [23]:
class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self,embed_dim,dense_dim,num_heads,**kwargs):
        super().__init__(**kwargs)

        self.num_heads = num_heads
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim

        self.mha = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim = embed_dim
        )
        self.dense_proj = tf.keras.Sequential(
            [
                tf.keras.layers.Dense(dense_dim,activation = 'relu'),
                tf.keras.layers.Dense(embed_dim)
            ]
        )
        self.layernorm_1 = tf.keras.layers.LayerNormalization()
        self.layernorm_2 = tf.keras.layers.LayerNormalization()
        self.supports_masking = True

    def call(self,inputs,mask = None):
        if mask is not None:
            padding_mask = tf.cast(mask[:,None,:],dtype = tf.int32)
        else:
            padding_mask = None
        attention_output = self.mha(
            query = inputs,
            value = inputs,
            key = inputs,
            attention_mask = padding_mask
        )
        proj_input = self.layernorm_1(attention_output+inputs)
        proj_output = self.dense_proj(proj_input)
        output = self.layernorm_2(proj_input + proj_output)
        return output
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim':self.embed_dim,
            'dense_dim':self.dense_dim,
            'num_heads': self.num_heads,
        })
        return config

In [24]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self,sequence_length,vocab_size,embed_dim,**kwargs):
        super().__init__(**kwargs)

        self.embed_dim = embed_dim
        self.vocab_size = vocab_size
        self.sequence_length = sequence_length
        
        self.token_embeddings = tf.keras.layers.Embedding(
            input_dim = vocab_size,
            output_dim  = embed_dim
        )

        self.position_embeddings = tf.keras.layers.Embedding(
            input_dim = sequence_length,
            output_dim = embed_dim
        )

    def call(self,inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start = 0,limit = length,delta =1)
        embeded_tokens = self.token_embeddings(inputs)
        embeded_position = self.position_embeddings(positions)
        return embeded_tokens + embeded_position
    
    def compute_mask(self,inputs,mask =None):
        if mask is not None:
            return tf.not_equal(inputs,0)
        else:
            return None
    def get_config(self):
        config = super().get_config()
        config.update({
            'vocab_size': self.vocab_size,
            'sequence_length': self.sequence_length,
            'embed_dim': self.embed_dim,
        })
        return config

In [25]:
class TransformerDecoder(tf.keras.layers.Layer):
    def __init__(self,embed_dim,latent_dim,num_heads,**kwargs):
        super().__init__(**kwargs)

        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads

        self.mha1 = tf.keras.layers.MultiHeadAttention(num_heads=num_heads,key_dim=embed_dim)
        self.mha2 = tf.keras.layers.MultiHeadAttention(num_heads = num_heads,key_dim=embed_dim)

        self.dense_proj = tf.keras.Sequential(
            [
                tf.keras.layers.Dense(latent_dim,activation = 'relu'),
                tf.keras.layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = tf.keras.layers.LayerNormalization()
        self.layernorm_2 = tf.keras.layers.LayerNormalization()
        self.layernorm_3 = tf.keras.layers.LayerNormalization()
        self.supports_masking = True

    def call(self,inputs,encoder_outputs,mask=None):
        casual_mask = self.get_casual_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:,None,:],dtype = tf.int32)
            padding_mask = tf.minimum(padding_mask,casual_mask)
        else:
            padding_mask = None

        attention_output1 = self.mha1(
            query = inputs,
            value = inputs,
            key = inputs,
            attention_mask = casual_mask
        )

        out_1 = self.layernorm_1(inputs + attention_output1)

        attention_output2 = self.mha2(
            query = out_1,
            value = encoder_outputs,
            key = encoder_outputs,
            attention_mask = padding_mask,
        )

        out_2 = self.layernorm_2(out_1 + attention_output2)
        proj_output = self.dense_proj(out_2)
        output = self.layernorm_3(proj_output + out_2)

        return output
    
    def get_casual_attention_mask(self,inputs):
        input_shape = tf.shape(inputs)
        batch_size,sequence_length = input_shape[0],input_shape[1]
        i = tf.range(sequence_length)[:,None]
        j = tf.range(sequence_length)
        mask = tf.cast(i>=j,tf.int32)
        mask = tf.reshape(mask,(1,input_shape[1],input_shape[1]))
        mult = tf.concat(
            [
                tf.expand_dims(batch_size,-1),
                tf.convert_to_tensor([1,1]),
            ],
            axis =0,
        )
        return tf.tile(mask,mult)
    

    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'latent_dim': self.latent_dim,
            'num_heads': self.num_heads
        })
        return config


In [27]:
# define emmbedding dimensions, latent dimensions, and number of heads
embed_dim = 100
latent_dim = 256
num_heads = 2

#Encoder
encoder_inputs = tf.keras.Input(shape = (None,), dtype = "int64", name = "encoder_inputs")

x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)

encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)

encoder = tf.keras.Model(encoder_inputs, encoder_outputs, name = "encoder")

#Decoder
decoder_inputs = tf.keras.Input(shape = (None,), dtype = "int64", name = "decoder_inputs")
encoder_seq_inputs = tf.keras.Input(shape = (None, embed_dim), name = "encoder_seq_inputs")

x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)

x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoder_seq_inputs)

x = tf.keras.layers.Dropout(0.5)(x)

decoder_outputs = tf.keras.layers.Dense(vocab_size, activation = tf.nn.log_softmax)(x)

decoder = tf.keras.Model([decoder_inputs, encoder_seq_inputs], decoder_outputs, name = "decoder")

# Define the final model
decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = tf.keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name = "transformer"
)


In [28]:
epochs = 1
losses = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
transformer.summary()

transformer.compile(
    "adam", loss = losses, metrics = ["accuracy"]
)
# transformer.fit(train_ds, epochs = epochs, validation_data = val_ds)


Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 100) │  1,502,000 │ encoder_inputs[0… │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, None, 100) │    132,656 │ positional_embed… │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder             │ (None, None,      │  3,230,556 │ decoder_inputs[0… │
│ (Functional)        │ 15000)            │            │ transformer_enco… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,865,212 (18.56 MB)

 Trainable params: 4,865,212 (18.56 MB)

 Non-trainable params: 0 (0.00 B)

In [29]:
epochs =20
transformer.fit(train_ds.repeat(),
                epochs = epochs,
                steps_per_epoch=500, 
                validation_data = val_ds.repeat(),
               validation_steps = 20,
               callbacks =tf.keras.callbacks.EarlyStopping(patience=3)
               )

Epoch 1/20


I0000 00:00:1727522032.458967      94 service.cc:145] XLA service 0x7cbe9c008b10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727522032.459023      94 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
W0000 00:00:1727522033.305982      94 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


 16/500 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.4701 - loss: 8.7711      

I0000 00:00:1727522038.543730      94 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


499/500 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6731 - loss: 3.4540

...

500/500 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - accuracy: 0.6733 - loss: 3.4492 - val_accuracy: 0.7682 - val_loss: 1.5082
Epoch 2/20
138/500 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.7701 - loss: 1.5483

W0000 00:00:1727522047.197722      94 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.7766 - loss: 1.4890 - val_accuracy: 0.8118 - val_loss: 1.1742
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8065 - loss: 1.2443 - val_accuracy: 0.8339 - val_loss: 0.9876
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8304 - loss: 1.0309 - val_accuracy: 0.8498 - val_loss: 0.8845
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8410 - loss: 0.9461 - val_accuracy: 0.8611 - val_loss: 0.7920
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8550 - loss: 0.8494 - val_accuracy: 0.8725 - val_loss: 0.7086
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8681 - loss: 0.7342 - val_accuracy: 0.8767 - val_loss: 0.6698
Epoch 8/20
498/500 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8721 - loss: 0.7061

W0000 00:00:1727522086.248184      93 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.8721 - loss: 0.7060 - val_accuracy: 0.8836 - val_loss: 0.6032
Epoch 9/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8820 - loss: 0.6315 - val_accuracy: 0.8872 - val_loss: 0.5740
Epoch 10/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8867 - loss: 0.5952 - val_accuracy: 0.8881 - val_loss: 0.5672
Epoch 11/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8878 - loss: 0.5849 - val_accuracy: 0.8859 - val_loss: 0.5848
Epoch 12/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8976 - loss: 0.5074 - val_accuracy: 0.8953 - val_loss: 0.5469
Epoch 13/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8959 - loss: 0.5169 - val_accuracy: 0.9004 - val_loss: 0.5004
Epoch 14/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8974 - loss: 0.5083 - val_accuracy: 0.8989 - val_loss: 0.5148
Epoch 15/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9042 - loss: 0.4550 - val_accuracy:

In [30]:
transformer.save_weights('english_french_model.weights.h5')

In [31]:
transformer.evaluate(test_ds)

170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9047 - loss: 0.4832


[0.4961366355419159, 0.9027403593063354]